# **AM 207**: Homework 5

Verena Kaynig-Fittkau and Pavlos Protopapas  <br>
**Due: 11.59 P.M. Thursday April 14th, 2016**

### Note: This homework is only for one week

### Instructions:

+ Upload your answers in an ipython notebook to Canvas.

+ We will provide you imports for your ipython notebook. Please do not import additional libraries.

+ Your individual submissions should use the following filenames: AM207_YOURNAME_HW5.ipynb

+ Your code should be in code cells as part of your ipython notebook. Do not use a different language (or format). 

+ **Do not just send your code. The homework solutions should be in a report style. Be sure to add comments to your code as well as markdown cells where you describe your approach and discuss your results. **

+ Please submit your notebook in an executed status, so that we can see all the results you computed. However, we will still run your code and all cells should reproduce the output when executed. 

+ If you have multiple files (e.g. you've added code files or images) create a tarball for all files in a single file and name it: AM207_YOURNAME_HW5.tar.gz or AM207_YOURNAME_HW5.zip


### Have Fun!
_ _ _ _ _

In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats 
import pandas as pd
import pymc as pm

import re
import numpy as np

import string

# Problem 1: HMM... I Think Your Text Got Corrupted!

In this problem you should use a Hidden Markov Model to correct typos in a text without using a dictionary. Your data is in two different text files:

* `Shakespeare_correct.txt` contains the words of some sonnets from Shakespeare
* `Shakespeare_typos.txt` contains the same text, but now some of the characters are corrupted

For convenience both text files only contain lower case letters a-z and spaces. 

First build a first order HMM:
* What are the hidden states and what are the observed states?
* What should you do to generate your HMM probability matrices?
* For some of the HMM parameters, you won't have enough training data to get representative probabilities.  For example, some of your probabilites might be 0. You should address this problem by adding a small pseudocount, similar to the motif finding problem from a previous assignment. 
* Implement the Viterbi algorithm and run it on a test portion that contains errors. Show that your Viterbi implementation can improve text of length 100, 500, 1000, and 2000. Note: To do this correctly you would have to withhold the part of the text that you use for testing when you estimate the parameters for you HMM. For the sake of this homework it is ok though to report training error instead of test error. Just be aware that the correction rate you are reporting most likely is a very optimistic estimate. 
* What correction rate do you get?

**Important**: Wikipedia has a nice article on [Viterbi](https://en.wikipedia.org/wiki/Viterbi_algorithm). **Please do not use the python implementation from this article!** (The lecture notebook also has the version from Wikipedia). Using dictionaries for Viterbi is really not intuitive and using numpy is typically faster. The article has very nice pseudo code that should enable you to easily program Viterbi by yourself. Please also refrain for this problem from using any other third party implementations. 

Now for a second order HMM:
By using a second order HMM, you should be able to get a better correction rate. 
* Give an intuitive explanation why a second order HMM should give better results.
* Implement your second order text correction. Hint: If you think a bit about the model you won't even have to change your Viterbi implementation. 
* Compare your correction rates against the first order model for text length of 100 and 500, (you can do 1000 as well if your computer is fast enough). 
* How well would your implementation scale to HMMs of even higher order? 

# Extra Problem 2: Final Project Review
    
You will be contacted shortly by a TF to meet and discuss your final project proposal. Be sure to take advantage of this feedback option. Review meetings should be scheduled within the week from April 11-15. 

In [14]:
#load in word sequences
with open('Shakespeare_correct.txt', 'r') as myfile:
    data = myfile.read() 
correct_l_len = len(data)
print 'length of correct letter is,', correct_l_len
correct_letters = data
#correct_words = data.split(' ')

with open('Shakespeare_typos.txt', 'r') as myfile:
    data=myfile.read()  
typo_len = len(data)
typo_letters = data
print 'length of typo letter is', typo_len
#typo_words = data.split(' ')




#function to calculate transition matrix of letters and spaces
def calc_1st_trans_prob(s1 = correct_letters):
    #initialize 
    ref_string = ' abcdefghijklmnopqrstuvwxyz'
    T = np.zeros((27,27),dtype = 'float')
    for i in range(1,len(s1)):
        row_zkm1 = ref_string.index(s1[i-1])
        col_zk = ref_string.index(s1[i])
        T[row_zkm1,col_zk] +=1
        
    #normalize and add pseudocount
    pseudocount = 0.1
    T += pseudocount
    
    for i in range(T.shape[0]):
        T[i,:] = T[i,:] / np.sum(T[i,:])
    
    return T

#calculate 2nd order transition matrix of letters and spaces
def calc_2nd_trans_prob(s1 = correct_letters):
    #initialize 
    ref_string = ' abcdefghijklmnopqrstuvwxyz'
    T = np.zeros((27**2,27**2),dtype = 'float')
    for i in range(2,len(s1)):
        row_zkm1_zkm2 = ref_string.index(s1[i-2]) * 27 + ref_string.index(s1[i-1])
        col_zk_zkm1 = ref_string.index(s1[i-1])*27 + ref_string.index(s1[i])
        T[row_zkm1_zkm2,col_zk_zkm1] +=1
        
    #normalize and add pseudocount
    pseudocount = 0.1
    T += pseudocount
    
    for i in range(T.shape[0]):
        T[i,:] = T[i,:] / np.sum(T[i,:])
    
    return T

def generate_2nd_HMM_states():
    states = ' abcdefghijklmnopqrstuvwxyz'
    hidden_states = [None] * (27**2)
    for i in range(len(states)):
        for j in range(len(states)):
            hidden_states[i*27 + j] = states[i] + states[j]
    

    return hidden_states

#function to generate second order observation inputs
def generate_2nd_HMM_obs(obs):
    obs_2nd_order = [None] * (len(obs)-1)
    for i in range(len(obs)-1):
        obs_2nd_order[i] = obs[i] + obs[i+1]

    return obs_2nd_order

#function to calculate emission probability matrix of letters and spaces
def calc_emission_prob(s1 = correct_letters, s2 = typo_letters):
    #initialize 
    ref_string = ' abcdefghijklmnopqrstuvwxyz'
    E = np.zeros((27,27),dtype = 'float')
    for i in range(0,len(s1)):
        row_zk = ref_string.index(s1[i])
        col_xk = ref_string.index(s2[i])
        E[row_zk,col_xk] +=1
        
    #normalize and add pseudocount
    pseudocount = 0.1
    E += pseudocount
    
    for i in range(E.shape[0]):
        E[i,:] = E[i,:] / np.sum(E[i,:])
    
    return E


#compute the 1st order transition matrix based on correct words
'''
T = np.zeros((num_correct_words,num_correct_words),dtype = 'float')
for i in range(1,len_sequence):
    row_zkm1 = sorted_correct_words.index(correct_words[i-1])
    col_zk   = sorted_correct_words.index(correct_words[i])
    T[row_zkm1,col_zk] += 1

#normalize and add pseudocount
for i in range(T.shape[0]):
    temp_sum = np.sum(T[i,:])
    if temp_sum > 0:
        T[i,:] = T[i,:]/temp_sum
    else:
        T[i,:] = 1.0/T.shape[0]
        #print 'here', 1/T.shape[0]
'''
#print T

# generate P(x|z) on spot!!
#compute the emission matrix 
#E = np.zeros(dtype='float')            

#function that calculates emission probability
#x: observed 
#z: hidden states
def calc_emit_prob(x,states):
    l = 0.01
    emit_prob = np.zeros(len(states),dtype ='float')
    for (indz,z) in enumerate(states):
        if len(x) == len(z):
            dist = len(x) - [s1 == s2 for (s1, s2) in zip(x, z)].count(True)
            emit_prob[indz] = l ** dist / scipy.misc.factorial(l)
        else:
            emit_prob[indz] = 0
    return emit_prob

#calculate correction rate
def calc_correction_rate(ref_list,typo_list,corrected_list):
    
    typo_match_rate = calc_match_rate(ref_list,typo_list)
    corr_match_rate = calc_match_rate(ref_list,corrected_list)
    
    return (corr_match_rate - typo_match_rate) / (1 - typo_match_rate)

#calculate match rate
def calc_match_rate(list_a,list_b):
    num_words = len(list_a)
    num_correct = 0;
    for i in range(num_words):
        if list_a[i] == list_b[i]:
            num_correct += 1

    return (num_correct + 0.0) / (num_words) +0.0
    
#numpy viterbi implementation:
def viterbi(obs, hidden_states,emission_states, start_p, trans_p, emit_p):
    
    #initialize:
    len_obs = len(obs)       #length of observation test string 
    num_states = len(hidden_states) #number of states, here 27 or 27**2
    V = np.zeros((len_obs,num_states),dtype = 'float') #log probabilities
    path = np.zeros((len_obs,num_states),dtype = 'int') -1
    
    #initialize base case
    '''
    for (indz,z) in enumerate(states):
        print indz, z
        print np.log(start_p[indz])
        print np.log(calc_emit_prob(obs[0],z))
        V[0,indz] = np.log(start_p[indz]) + np.log(calc_emit_prob(obs[0],z))
        path[0,indz] = indz
    '''
    V[0,:] = np.log(start_p) + np.log(emit_p[:,emission_states.index(obs[0])].transpose())
    '''
    incr = num_states / len(emission_states)
    for i in range(len(emission_states)):
        V[0,(i*incr):((i+1)*incr)] += np.log(emit_p[i,emission_states.index(obs[0])])
    '''
    #path[0,:] = np.arange(0,num_states)
    
    #print 'starting...', np.log(calc_emit_prob(obs[0],states))
    
    print V.shape
    #assert(0)
    
    #run viterbi for t>0 
    for t in range(1,len(obs)):
        #print t
        #log_emit_prob = np.log(calc_emit_prob(obs[t],states))
        log_emit_prob = np.log(emit_p[:,emission_states.index(obs[t])].transpose())
        #print 't =' ,t ,', log_emit_prob=' ,log_emit_prob
        
        for (indz, z) in enumerate(emission_states):

            #log_prob_vec = V[t-1,:] + np.log(trans_p[:,indz].transpose()) + np.log(calc_emit_prob(obs[t],states))
            log_prob_vec = V[t-1,:] + np.log(trans_p[:,indz].transpose()) + log_emit_prob[indz]
            V[t,indz] = np.max(log_prob_vec)
            path[t-1,indz] = np.argmax(log_prob_vec)
            
            #print 'z= ', z, ',transition prob =', trans_p[indz,:].transpose(), \
            #      ',emission prob = ', np.exp(log_emit_prob[indz]), ',path=', path, \
            #      ',V[t,indz] =', V[t,indz]
    
    #find best solution based on IC
    #print 'writing solution...\n'
    best_ind = np.argmax(V[t,:])
    path[t,best_ind] = best_ind
    #print 'best indices is: ', best_ind
    #print 'final path =', path
    best_log_prob = V[t,best_ind]
    #best_path_ind = path[:,best_ind]
    
    #best_path = [states[best_ind]]
    ##best_path = hidden_states[best_ind]
    best_path = emission_states[best_ind]
    
    col_ind = best_ind
    for i in range(len(obs)-2,-1,-1):
        col_ind = path[i,col_ind]
        #best_path.append(states[col_ind])
        ##best_path += hidden_states[col_ind]
        best_path += emission_states[col_ind]
    
    #best_path = best_path[-1::-len(hidden_states[0])]
    best_path = best_path[::-1]
    #print best_path
    #print 'V=', V
    #print best_log_prob
    #print 'best_path =', best_path
    #print 'obs =', obs
    return best_log_prob, best_path

#testing
#print a_vec.index('a')


#driver script
if __name__ == '__main__':
    print '1st order HMM starts ...\n'
    
    #generate 1st order transition matrices
    T_1st = calc_1st_trans_prob()
    #print T_1st
    
    #generate 2nd order transition matrices
    T_2nd = calc_2nd_trans_prob()
    
    E = calc_emission_prob()
    #print E
    
    
    #1st order HMM
    
    hidden_1st_states = list(' abcdefghijklmnopqrstuvwxyz')
    emission_states = ' abcdefghijklmnopqrstuvwxyz'
    start_p = np.ones(len(emission_states),dtype = 'float')
    best_log_prob, best_path = viterbi(typo_letters[0:100],hidden_1st_states,emission_states,start_p,T_1st,E)
    correct_rate = calc_correction_rate(correct_letters[0:100],typo_letters[0:100],best_path)
    print 'correction rate =', correct_rate, '\n typo list =', typo_letters[0:100], '\n corrected list =', \
          best_path, '\n true list=', correct_letters[0:100]
       
    #2nd order HMM
    #print generate_2nd_HMM_states()
    
    print '\n2nd order HMM starts ...'
    hidden_2nd_states = generate_2nd_HMM_states()
    emission_states = generate_2nd_HMM_states()
    obs_2nd_order = generate_2nd_HMM_obs(typo_letters[0:101])
    print obs_2nd_order
    #emission_states = ' abcdefghijklmnopqrstuvwxyz'
    start_p = np.ones(len(emission_states)**2,dtype = 'float')
    best_log_prob, best_path = viterbi(obs_2nd_order,hidden_2nd_states,emission_states,start_p,T_2nd,E)
    print 'full hidden states =', best_path
    best_path = best_path[0::2]
    correct_rate = calc_correction_rate(correct_letters[0:100],typo_letters[0:100],best_path)
    print 'correction rate =', correct_rate, '\n typo list =', typo_letters[0:100], '\n corrected list =', \
          best_path, '\n true list=', correct_letters[0:100]
    
    
    #start_p = np.ones(len(sorted_correct_words),dtype = 'float')
    #print 'typo words are, ', typo_words[0:10]
    #print 'states are', sorted_correct_words
    #best_log_prob, best_path = viterbi(typo_words[100:200],sorted_correct_words,start_p,T)
    #correct_rate = calc_correction_rate(best_path,correct_words[100:200])
    #print 'correction rate =', correct_rate, '\n corrected list =', \
    #      best_path, '\n original list=', correct_words[100:200]



#testing code
'''
a = ["asd","def","ase","dfg","asd","def","dfg"]
a = list(set(a))
b = sorted(a)
print a, '\n', b

import operator
print dist , dist2

print 'emission prob', calc_emit_prob('abc',['abc','asd'])
print 'emission prob', calc_emit_prob('abc',['abc','asdc'])
print 'emission prob', calc_emit_prob('abc',['acb','asd'])
print 'emission prob', calc_emit_prob('abc',['abe','asd'])
print b.index("def")

v= []
for i in range(10):
    v.append('a')
print v

for (inda,suba) in enumerate(a):
    print inda, suba


a_vec = ['asd','sda','sda']
b_vec = ['asd','ccc','sdc']


print calc_correction_rate(a_vec,b_vec)
'''

#print num_correct



length of correct letter is, 60208
length of typo letter is 60208
1st order HMM starts ...

(100, 27)
correction rate = 0.6 
 typo list = fron fbirest crebtuses we eesjsf iocrease uhat therfbz bebuuys rose night never eie buu as uie siper 
 corrected list = fron fairest breatuses we desise increase that thereay beautys rore might mever die but as the siper 
 true list= from fairest creatures we desire increase that thereby beautys rose might never die but as the riper

2nd order HMM starts ...
['fr', 'ro', 'on', 'n ', ' f', 'fb', 'bi', 'ir', 're', 'es', 'st', 't ', ' c', 'cr', 're', 'eb', 'bt', 'tu', 'us', 'se', 'es', 's ', ' w', 'we', 'e ', ' e', 'ee', 'es', 'sj', 'js', 'sf', 'f ', ' i', 'io', 'oc', 'cr', 're', 'ea', 'as', 'se', 'e ', ' u', 'uh', 'ha', 'at', 't ', ' t', 'th', 'he', 'er', 'rf', 'fb', 'bz', 'z ', ' b', 'be', 'eb', 'bu', 'uu', 'uy', 'ys', 's ', ' r', 'ro', 'os', 'se', 'e ', ' n', 'ni', 'ig', 'gh', 'ht', 't ', ' n', 'ne', 'ev', 've', 'er', 'r ', ' e', 'ei', 'ie', 'e ', ' 

IndexError: index 180 is out of bounds for axis 1 with size 27